In [1]:
# import
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
from datetime import datetime
import pandas as pd
import time
import random
import os
import sqlite3

In [2]:
# 주피터 파일이랑 같은 폴더에 chromedriver 넣어야합니다.
driver = webdriver.Chrome('chromedriver.exe')
soup = bs(driver.page_source, 'html.parser')
driver.implicitly_wait(3)

# 이렇게 입력할 수 도 있고, 텍스트파일로 만들어둬서 입출력 자동화해도되구~
#swarm_ID=input("ID를 입력해주세요 : ")
#swarm_PW=input("PW를 입력해주세요 : ")
swarm_ID = 'his9989@naver.com'
swarm_PW = 'helloworld'

# swarm login url 접속
driver.get('https://ko.swarmapp.com/login')

# ID, PW 텍스트박스에 정보 입력
driver.find_element_by_name('emailOrPhone').send_keys(swarm_ID)
driver.find_element_by_name('password').send_keys(swarm_PW)

#로그인 버튼 클릭
driver.find_elements_by_css_selector("p.loginOrSignup > input")[0].click()

In [3]:
# 경로 목록으로 들어가는 작업.
# 이거 안하면 엉뚱한거 뽑힘
driver.find_elements_by_css_selector("div.user-arrow > img")[0].click()
driver.find_elements_by_css_selector("li > a.userHistoryLink")[0].click()

In [4]:
from selenium.webdriver.common.keys import Keys
element=driver.find_element_by_tag_name('html')
now_len = 0

while 1:
    # 50개씩만 보이고, 스크롤 내려가면 추가적으로 보이는 구조
    # 스크롤을 끝까지 내려서 모든 데이터가 보여야해서 만든 코드
    # 근데 이부분이 살짝 엉킬수도있음..
    # 250 좀 조정하면서 해보고
    # 끝까지 안내려갔는데 그냥 finish되면 그냥 손으로 스크롤 내리셈
    for j in range(250):
        element.send_keys(Keys.ARROW_DOWN)    
    #time.sleep(3)
    locate_datas = driver.find_elements_by_css_selector('div.activity')
    if now_len == len(locate_datas):
        print('finish')
        break
    now_len = len(locate_datas)
    print('now_len : ' + str(now_len))

now_len : 100
now_len : 150
now_len : 200
now_len : 300
now_len : 350
now_len : 400
now_len : 405
finish


In [5]:
# n일전을 계산하기 위한, 코드 수행 순간 날짜
now = datetime.now()
now_str = str(now.year)+"년 "+str(now.month)+"월 "+str(now.day)+"일"
swarm_ID = 'his9989@naver.com'
print(now_str)
output_data = swarm_ID+'_'+now_str
conn_o = sqlite3.connect(output_data)
curs_o = conn_o.cursor()

try:
    curs_o.execute("create table position(location, location_type, time)")
except:
    print('already exists')
curs_o.execute('delete from position')

2019년 3월 7일


In [12]:
soup = bs(driver.page_source, 'html.parser')
curs_o = conn_o.cursor()

# 실질적인 수집부
for j in range(now_len):
    print(str(j+1)+"..", end=" : ")
    
    location = soup.select('span.activity-checkinInfo-venue > a')[j].get_text()
    location_type = soup.select('p > span.global')[j].get_text()
    
    print(location, end=", ")
    print(location_type, end=", ")
    
    try:
        curs_o.execute("SELECT time FROM position WHERE location = '"+location+"'")
        count_temp = curs_o.fetchall()
    except:
        continue
    
    try:
        count_t = count_temp[0][0]
        count = count_t + 1
    except:
        count = 1
    
    if count != 1:
        values = [(location, location_type, count)]
        curs_o.execute("UPDATE position SET time = "+str(count) + " WHERE location = '" + location + "'")
        update_complete = curs_o.fetchall()
        
    elif count == 1:
        values = [(location, location_type, 1)]
        curs_o.executemany('insert into position values(?, ?, ?)', values)
        conn_o.commit()
    print(values)
conn_o.close()
print('finish..')

1.. : 홍익대학교 제4공학관 T동 819호 (PEM), 서강동, [('홍익대학교 제4공학관 T동 819호 (PEM)', '서강동', 10)]
2.. : Chop Hair Salon, 서교동, [('Chop Hair Salon', '서교동', 3)]
3.. : 한양대역 육교, 사근동, [('한양대역 육교', '사근동', 7)]
4.. : 하나은행 (HANA BANK), 동대문구, [('하나은행 (HANA BANK)', '동대문구', 3)]
5.. : 신락원, 동대문구, [('신락원', '동대문구', 4)]
6.. : 서울시립대 정문, 동대문구, [('서울시립대 정문', '동대문구', 3)]
7.. : 래미안크레시티, 전농1동, [('래미안크레시티', '전농1동', 7)]
8.. : 병천 아우내 순대, 동대문구, [('병천 아우내 순대', '동대문구', 3)]
9.. : 우동가조쿠, 사근동, [('우동가조쿠', '사근동', 3)]
10.. : 마호가니, 잠실6동, 송파구, [('마호가니', '잠실6동, 송파구', 4)]
11.. : 한양대역 육교, 사근동, [('한양대역 육교', '사근동', 8)]
12.. : 삼성SDS타워 서관, 잠실6동, [('삼성SDS타워 서관', '잠실6동', 2)]
13.. : 뺑드미, 전농1동, [('뺑드미', '전농1동', 4)]
14.. : 스시도쿠, 행당1동, [('스시도쿠', '행당1동', 2)]
15.. : 동키치킨, 잠실6동, [('동키치킨', '잠실6동', 4)]
16.. : 버거킹, 화양동, [('버거킹', '화양동', 2)]
17.. : 왕십리광장, 행당1동, [('왕십리광장', '행당1동', 3)]
18.. : 행운돈까스, 사근동, [('행운돈까스', '사근동', 3)]
19.. : FOREVER 21, 서교동, [('FOREVER 21', '서교동', 2)]
20.. : ABC Mart, 서교동, [('ABC Mart', '서교동', 4)]
21.. : 서교동 주민센터, 서교동, [('서교동 주민센터', '서교동

151.. : bogner coffee, 전농1동, [('bogner coffee', '전농1동', 1)]
152.. : 래미안크레시티, 전농1동, [('래미안크레시티', '전농1동', 15)]
153.. : Starbucks, 종로1.2.3.4가동, [('Starbucks', '종로1.2.3.4가동', 1)]
154.. : 이삭토스트, 종로1.2.3.4가동, [('이삭토스트', '종로1.2.3.4가동', 10)]
155.. : 한양대역 육교, 사근동, [('한양대역 육교', '사근동', 13)]
156.. : angel-in-us specialty, 종로1.2.3.4가동, [('angel-in-us specialty', '종로1.2.3.4가동', 3)]
157.. : Subway, 종로1.2.3.4가동, [('Subway', '종로1.2.3.4가동', 4)]
158.. : 굽네치킨, 종로1.2.3.4가동, [('굽네치킨', '종로1.2.3.4가동', 1)]
159.. : 이삭토스트, 종로1.2.3.4가동, [('이삭토스트', '종로1.2.3.4가동', 11)]
160.. : 한양대역 육교, 사근동, [('한양대역 육교', '사근동', 14)]
161.. : JS Kitchen, 잠실6동, [('JS Kitchen', '잠실6동', 1)]
162.. : 버거킹, 서교동, [('버거킹', '서교동', 3)]
163.. : 본죽, 서교동, [('본죽', '서교동', 1)]
164.. : 홍익대학교 제4공학관 T동 819호 (PEM), 서강동, [('홍익대학교 제4공학관 T동 819호 (PEM)', '서강동', 19)]
165.. : Ahndamiro Italian Restaurant, 인사동, [('Ahndamiro Italian Restaurant', '인사동', 1)]
166.. : 굽네치킨, 종로1.2.3.4가동, [('굽네치킨', '종로1.2.3.4가동', 2)]
167.. : Starbucks, 종로1.2.3.4가동, [('Starbucks', '종로1.

305.. : 홍익대학교 학생회관, 서강동, [('홍익대학교 학생회관', '서강동', 3)]
306.. : 홍익대학교 제2기숙사, 서강동, [('홍익대학교 제2기숙사', '서강동', 9)]
307.. : 경복궁, 송파구, [('경복궁', '송파구', 1)]
308.. : 홈플러스, 잠실6동, [('홈플러스', '잠실6동', 2)]
309.. : 홍익대학교 제4공학관 T동 819호 (PEM), 서강동, [('홍익대학교 제4공학관 T동 819호 (PEM)', '서강동', 30)]
310.. : 홍익대학교 인문사회관 C동, 서강동, [('홍익대학교 인문사회관 C동', '서강동', 3)]
311.. : The Coffee Bean & Tea Leaf, 서교동, [('The Coffee Bean & Tea Leaf', '서교동', 1)]
312.. : 오렌지약국, 잠실6동, [('오렌지약국', '잠실6동', 10)]
313.. : 탐라상회, 송파구, 서울특별시, [('탐라상회', '송파구, 서울특별시', 1)]
314.. : 라인프렌즈, 서교동, [('라인프렌즈', '서교동', 1)]
315.. : 빽다방 (Paik's Coffee), 송파구, 316.. : A Twosome Place, 송파구, [('A Twosome Place', '송파구', 1)]
317.. : 사루식당, 잠실6동, [('사루식당', '잠실6동', 3)]
318.. : ABC Mart, 서교동, [('ABC Mart', '서교동', 12)]
319.. : 홍익대학교 제4공학관 T동 819호 (PEM), 서강동, [('홍익대학교 제4공학관 T동 819호 (PEM)', '서강동', 31)]
320.. : 내가찜한닭, 서강동, [('내가찜한닭', '서강동', 1)]
321.. : 홍익대학교 제4공학관 T동 819호 (PEM), 서강동, [('홍익대학교 제4공학관 T동 819호 (PEM)', '서강동', 32)]
322.. : Mom's Touch, 서강동, 323.. : 잠실철교, 구의3동, [('잠실